### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 27 апреля 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 27 апреля, -4 балла после 08:30 4 мая, -6 баллов после 08:30 11 мая, -8 баллов после 08:30 18 мая.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import tqdm

In [101]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __err(self, y, count):
        A = np.max(y, axis=1) / count
        A[np.isnan(A)] = 0
        return 1 - A

    def __gini(self, y, count):
        A = y / count[:, np.newaxis]
        A[np.isnan(A)] = 0
        return 1 - np.sum(A ** 2, axis=1)
    
    def __entropy(self, y, count):
        A = y / count[:, np.newaxis]
        probas = A
        A[A == 0] = 1
        return -1 * np.sum(probas * np.apply_along_axis(lambda x: np.log2(x), 0, probas), axis=1)
        
    
    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        index = x.argsort()
        x_s = x[index]
        y_s = y[index]
        thresholds, potential_splits = np.unique(x_s, return_index=True)
        y_l = np.zeros((potential_splits.shape[0], self.num_class))
        y_r = np.zeros((potential_splits.shape[0], self.num_class))
        for i in range(y_l.shape[0]):
            y_l[i] = np.bincount(y_s[:potential_splits[i]], minlength=self.num_class)
            y_r[i] = np.bincount(y_s[potential_splits[i]:], minlength=self.num_class)
        count_l = np.sum(y_l, axis=1)
        count_r = np.sum(y_r, axis=1)
        y_s = y_l + y_r
        count = count_l + count_r
        if self.criterion == 'err':
            foo = self.__err
        elif self.criterion == 'gini':
            foo = self.__gini
        else:
            foo = self.__entropy
        gain = foo(y_s, count) - (count_l * foo(y_l, count_l) + count_r * foo(y_r, count_r)) / count
        idx = np.argmax(gain)
        return gain[idx], thresholds[idx]
        
    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        if (depth >= self.max_depth) or (y.shape[0] < self.min_samples_split) or (np.unique(y).shape[0] == 1):
            binc = np.bincount(y)
            prob = binc.max() / y.shape[0]
            pred = binc.argmax()
            self.tree[node_id] = (self.LEAF_TYPE, pred, prob)     
        else:
            A = np.array([self.__find_threshold(x[:, feature_id], y) for feature_id in range(x.shape[1])])
            id = np.argmax(A[:, 0])
            thres = A[id, 1]
            self.feature_importances_[id] += A[id, 0]
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, id, thres)
            if (x_l.shape[0] == 0) or (x_r.shape[0] == 0):
                binc = np.bincount(y)
                prob = binc.max() / y.shape[0]
                pred = binc.argmax()
                self.tree[node_id] = (self.LEAF_TYPE, pred, prob)
            else:
                self.tree[node_id] = (self.NON_LEAF_TYPE, id, thres)
                self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
                self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 2)
        
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0) 
        self.feature_importances_ /= np.sum(self.feature_importances_)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self):
        """
        Возвращает важность признаков
        """
        return self.feature_importances_

In [102]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [103]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [104]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [105]:
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test), accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

(0.8888888888888888, 1.0)

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [109]:
%time clf.fit(X_train, y_train)

Wall time: 2.16 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [110]:
%time my_clf.fit(X_train, y_train)

Wall time: 32.9 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [16]:
data = pd.read_csv('./data/Speed Dating Data.csv', encoding='latin1').iloc[:, :97].drop(['id', 'idg', 'round', 'position', 'positin1', 'order', 'partner', 'from', 'zipcode', 'field', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin','pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o','met_o', 'undergra', 'income', 'tuition', 'mn_sat', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga', 'career'], axis=1)

In [17]:
for i in [3, 4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    data = data.drop(feat, axis=1)
data = data.dropna(subset=['age'])
data = data.dropna(subset=['imprelig', 'imprace'])

In [18]:
data.loc[:, 'career_c'] = data.loc[:, 'career_c'].fillna(59)
data.loc[:, 'field_cd'] = data.loc[:, 'field_cd'].fillna(19)
data = data.reset_index(drop=True)

In [19]:
encoder = OneHotEncoder(sparse=False)

A = encoder.fit_transform(data.race.values.reshape(-1, 1))
B = pd.DataFrame(A, columns=['race_' + str(i) for i in range(A.shape[1])])
data = pd.concat([data, B], axis=1).drop(['race'], axis=1)

A = encoder.fit_transform(data.career_c.values.reshape(-1, 1))
B = pd.DataFrame(A, columns=['career_' + str(i) for i in range(A.shape[1])])
data = pd.concat([data, B], axis=1).drop(['career_c'], axis=1)

A = encoder.fit_transform(data.field_cd.values.reshape(-1, 1))
B = pd.DataFrame(A, columns=['field_' + str(i) for i in range(A.shape[1])])
data = pd.concat([data, B], axis=1).drop(['field_cd'], axis=1)

In [20]:
X = np.array(data.drop(['match', 'iid', 'pid'], axis=1))
y = np.array(data.match)

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [37]:
cv = KFold(n_splits=20, shuffle=True)
scores = []
criterions = ['err', 'gini', 'entropy']
depths = range(1, 11)
min_samples = range(2, 11)
with tqdm(total=20) as pbar:
    for train_index, test_index in cv.split(y):
        accuracy = np.empty((3, 10, 9))
        y_train, y_test = y[train_index], y[test_index]
        X_train, X_test = X[train_index], X[test_index]
        for criterion in range(3):
            for depth in depths:
                for min_sample in min_samples:
                    clf = MyDecisionTreeClassifier(min_sample, depth, criterions[criterion])
                    clf.fit(X_train, y_train)
                    accuracy[criterion, depth - 1, min_sample - 2] = accuracy_score(clf.predict(X_test), y_test)
        scores.append(accuracy)
        pbar.update()

In [78]:
total_score = np.zeros((3, 10, 9))
for i in range(20):
    total_score += scores[i]
total_score /= 20
total_score.max()

0.8354876536711467

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [114]:
clf = MyDecisionTreeClassifier(1, 10, 'gini')
clf.fit(X, y)

In [119]:
df = pd.DataFrame()
df['num'] = np.argsort(clf.get_feature_importance())[-10:][::-1]
df['name'] = data.drop(['match', 'iid', 'pid'], axis=1).columns[df['num']]
df

,num,name
0,3,int_corr
1,5,age
2,4,samerace
3,6,imprace
4,12,expnum
5,9,date
6,24,shar2_1
7,1,condtn
8,20,sinc2_1
9,18,shar1_1


## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

